In [1]:
# trimmik

In [2]:
import argparse
import numpy as np
from Bio import SeqIO



In [3]:
parser = argparse.ArgumentParser(description='Name of programe')
parser.add_argument('-hc', '--headcrop', help='Number of cropped head sequence', metavar='Int', type=int, required=False)
parser.add_argument('-tc', '--tailcrop', help='Nimber of cropped tail sequence', metavar='Int', type=int, required=False)
parser.add_argument('-w', '--window', help='The size of the window that will be used for quality evaluation', metavar='Int', type=int)
parser.add_argument('-thr', '--threshold', help='The sthreshold for quality evaluation', metavar='Int', type=int)
args = parser.parse_args(args="-hc 5 -tc 10 -w 10 -thr 20".split(" "))



In [38]:
def trim(args, rec):
    phred_qual = list(rec.letter_annotations["phred_quality"])
    rec.letter_annotations = {}
    
    if args.headcrop:
        rec.seq = rec.seq[args.headcrop:]
        phred_qual = phred_qual[args.headcrop:]
        
    if args.tailcrop:
        rec.seq = rec.seq[:args.tailcrop]
        phred_qual = phred_qual[:args.tailcrop]
           
    if args.window and args.threshold:
        start = 0 
        seq = rec.seq.tomutable()
        while start <= len(seq)-args.window:
            if np.mean(phred_qual[start:start+args.window]) < args.threshold:
                for i in range(start, start+args.window):
                    del seq[start]
                phred_qual[start:start+args.window] = []
            else:
                start += 1
        rec.seq = seq   
    rec.letter_annotations["phred_quality"] = phred_qual
    return rec

reads = SeqIO.parse("./test_classwork2.fastq", "fastq")
result = [trim(args,rec) for rec in reads]
SeqIO.write(result, "trimmed.fastq", "fastq") 
        

28160